# 3. Bistability through positive feedback

<hr>

**Design principles**

- Positive, ultrasensitive feedback loops enable bistability.
- Hysteresis (history-dependent system behavior) allows cells to encode memory of past events. 

**Concepts**

- Bifurcations are critical points where small changes in a parameter cause large changes in system behavior.

**Techniques**

- Graphical approaches for characterizing state stability 
- Nullcline analysis identifies fixed points and provides insight into the structure of dynamical systems.
- Nondimensionalization simplifies analysis of a model's parameter space.

<hr>

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://biocircuits.github.io/chapters/data/"
else:
    data_path = "data/"
# ------------------------------

import numpy as np
import pandas as pd
import scipy.integrate
import scipy.optimize

import biocircuits.jsplots

import bokeh.io
import bokeh.models
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

## Positive autoregulation enables bistability

One of the most amazing aspects of living cells is their ability to exist in different—sometimes radically different—states. In animals, neurons look and behave quite differently than skin or blood cells even though they all share the same genome. Single cell analysis and [large scale cell atlas projects](https://doi.org/10.7554/eLife.27041) are only beginning to reveal the full diversity of cell states. And, as we zoom in to examine individual cells of any one nominal type, we often find many sub-types, with different morphologies, functions, spatial locations, and gene expression states. Something similar occurs in bacteria, where a given strain can often exist in different states that differ in function and gene expression. (In fact, it is not easy to define [what we mean by a cell type](https://doi.org/10.1016/j.cels.2017.03.006)!) From the circuit point of view, a fundamental question is: What types of circuits can enable an individual cell to exist in two or more distinct states, each one stable on its own?

The key circuit property we will study is **bistability**, defined as the ability of a circuit to exist in two distinct stable states. We will see that bistability can be achieved with circuits as simple as a single gene, as long as they possess two key properties: positive feedback and ultrasensitivity.

In Chapter 1, we analyzed the steady-state behavior of a gene regulated by an activator, which we assumed was present at some fixed concentration within the cell. However, just as repressors can repress their own expression (negative autoregulation), as we saw in the Chapter 2, activators can activate their own expression. This phenomenon, **positive autoregulation**, occurs with both bacterial and eukaryotic transcription factors, especially those that establish and maintain cell states. Here, we will show how positive autoregulation can, under some conditions, generate bistability. 

<div style="width: 200px; margin: auto;">

![positive autoregulation](figs/positive_autoregulation_2.png)

</div>

We can represent a positive autoregulatory circuit with the following ODE, where the production term for the activator, X, is an activating Hill function of its own concentration, $x$:

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t} = \beta\, \frac{(x/k)^n} {1 + (x/k)^n} - \gamma x .
\end{align}

We can get some intuition about this system's behavior without diving straight into solving the ODE or even finding its steady states. First, we notice that $\mathrm{d}x/\mathrm{d}t$ vanishes for $x = 0$, regardless of the parameter values, so $x = 0$ is always a steady state. If the activator is absent, it stays absent.

To go further, we will turn to graphical methods and plot the values of the production term (the activating Hill function) and the degradation term ($\gamma x$) for various values of $x$.

In [2]:
# Parameters
beta = 10
gamma = 1
k = 4
n = 4

# Theroetical curves
x = np.linspace(0, 20, 400)
prod = beta * (x / k) ** n / (1 + (x / k) ** n)
removal = gamma * x

# Fixed points
fp_rhs = lambda x: beta / gamma * (x / k) ** n / (1 + (x / k) ** n)
fixed_points = [
    0,
    float(scipy.optimize.fixed_point(fp_rhs, 3)),
    float(scipy.optimize.fixed_point(fp_rhs, 5)),
]

# Build plot
p = bokeh.plotting.figure(
    frame_height=275,
    frame_width=375,
    x_axis_label="x",
    y_axis_label="production or removal rate",
    title=f"β = {beta}, γ = {gamma} , k = {k}, n = {n}",
    x_range=[-1, 20],
)

# Plot production and removal rates
p.line(x, prod, line_width=2, color="#1f77b4")
p.line(x, removal, line_width=2, color="orange")

# Plot fixed points
for i, fp in enumerate(fixed_points):
    if i % 2 :
        p.add_layout(
            bokeh.models.Span(
                location=fp,
                level="underlay",
                dimension="height",
                line_color="black",
                line_dash="dashed",
            )
        )
        fill_color = "white"
    else:
        fill_color = "black"

    p.circle(
        [fp], [gamma * fp], color="black", size=10, line_width=2, fill_color=fill_color,
    )

# Annotate
p.add_layout(
    bokeh.models.Arrow(
        end=bokeh.models.VeeHead(size=15, fill_color="gray", line_color="gray"),
        line_width=4,
        x_start=3.25,
        y_start=12,
        x_end=0.1,
        y_end=12,
        line_color="gray",
    )
)
p.add_layout(
    bokeh.models.Arrow(
        end=bokeh.models.VeeHead(size=15, fill_color="gray", line_color="gray"),
        line_width=4,
        x_start=3.5,
        y_start=12,
        x_end=6.75,
        y_end=12,
        line_color="gray",
    )
)
p.add_layout(
    bokeh.models.Arrow(
        end=bokeh.models.VeeHead(size=15, fill_color="gray", line_color="gray"),
        line_width=4,
        x_start=20,
        y_start=12,
        x_end=13,
        y_end=12,
        line_color="gray",
    )
)
p.text(
    x=[14],
    y=[9.933],
    text=["production rate"],
    text_color="#1f77b4",
    text_font_size="10pt",
    text_align="left",
    text_baseline="top",
)
p.text(
    x=[14],
    y=[14],
    text=["removal rate"],
    text_color="orange",
    text_font_size="10pt",
    text_align="left",
    angle=0.6,
)

bokeh.io.show(p)

The value of $\mathrm{d}x/\mathrm{d}t$ at a particular value of $x$ is determined by the difference between the production rate and the degradation rate for that $x$ value. Whenever the production rate is equal to the degradation rate, the time derivative vanishes ($\mathrm{d}x/\mathrm{d}t = 0$), and we have a **fixed point**. 

We can see in the plot above that, for the parameters that were chosen here, there are three fixed points corresponding to the three intersections between the production rate function and the removal rate function. This means that there are three possible values of $x$ that give steady states. How do we know which fixed point the system will move toward?

We can answer this question by determining the **stability** of each fixed point. Notice that the above graph, in addition to telling us the location of the fixed points, also tells us the direction that the system will move at any given value of $x$. If, at that value, the production rate is higher than the degradation rate, then $\mathrm{d}x/\mathrm{d}t > 0$ so $x$ will increase. Correspondingly, if the degradation rate is higher than the production rate at a particular value of $x$, then $\mathrm{d}x/\mathrm{d}t < 0$ and $x$ will decrease. We have marked this information in the above plot with horizontal arrows indicating the direction that the system when $x$ lies within different ranges.

Looking at these arrows, we see that the system moves towards some fixed points and away from others. When a system near a fixed point tends to move towards that fixed point, from any direction, then we call that fixed point **stable**. Otherwise, it is called an **unstable** fixed point. Unless this system's initial condition places it exactly at the unstable fixed point, as the system evolves over time it will converge to one of the two stable fixed points. Which fixed point it converges to is determined by the initial value of $x$; if the system starts below the unstable fixed point (to the left of the dashed line) then it will converge to the lower stable fixed point; if it starts above the unstable fixed point it will converge to the higher stable fixed point. In such a way, each stable fixed point has a **basic of attraction**, a set of values of $x$ for which the system will converge toward the fixed point. In the above plot, the basin of attraction for the higher fixed point is all $x$-values to the right of the dashed line, and that for the lower fixed point is all $x$-value to the left of the dashed line (including, of course, only non-negative $x$).

Since this system has two stable fixed points, we can describe it as **bistable**. As long as noise or other perturbations are not too strong, the cell can happily remain at either a low or high value of x. Bistability is a special case of the more general phenomenon of **multistability**, which is one of the most important properties in biology, underlying the ability of a single genome to produce a vast array of distinct cell types in a multicellular organism. (We will discuss a larger multistable circuit later in the course.) 

This simple analysis shows us that a single positive autoregulatory gene can generate bistability! However, positive feedback by itself _is not enough_. Bistabiity also requires an **ultrasensitive** response to $x$. (In the context of the Hill function, ultrasensitivity can be defined as $n>1$). If we reduce the Hill coefficient to 1, keeping other parameters the same, you can see that we now have only a single stable fixed point (and one unstable fixed point at $x=0$).

In [3]:
# Reduce n to 1
n = 1

# Recompute production curve
prod = beta * (x/k) ** n / (1 + (x/k) ** n)

# Fixed points
fp_rhs = lambda x: beta / gamma * (x/k) ** n / (1 + (x/k) ** n)
fixed_points = [0, float(scipy.optimize.fixed_point(fp_rhs, 3))]

# Build plot
p = bokeh.plotting.figure(
    frame_height=275,
    frame_width=375,
    x_axis_label="x",
    y_axis_label="production or removal rate",
    title=f"β = {beta}, γ = {gamma} , k = {k}, n = {n}",
    x_range=[-1, 20],
)

# Plot production and removal rates
p.line(x, prod, line_width=2, color="#1f77b4")
p.line(x, removal, line_width=2, color="orange")

# Plot fixed points
for i, fp in enumerate(fixed_points):
    fill_color = "black" if i % 2 else "white"
    p.circle(
        [fp], [gamma * fp], color="black", size=10, line_width=2, fill_color=fill_color,
    )

bokeh.io.show(p)

In an end-of-chapter problem, you can prove that no matter how we change the other parameters in this model, the system will never be able to obtain bistability unless it has ultrasensitivity ($n>1$). 

Nevertheless, ultrasensitivity does *not* guarantee bistability in this system. It also requires tuning of the system's other parameters relative to one another. We can explore this relationship between the parameter values in the interactive plot below.

In [4]:
bokeh.io.show(biocircuits.jsplots.autoactivator_fixed_points())

As $\gamma$ increases, the upper two fixed points get closer and closer together, eventually disappearing as the system becomes monostable, always converging to the single value of $x=0$, and therefore unable to activate itself. The range of parameter values that is near this disappearing point is important because a very small change in a parameter can lead to a very drastic change in the behavior of the system. Such a phenomenon is called a **bifurcation**. In this case, we can see that the system transitions between monostability and bistability, and within a critical region of parameter space near this bifurcation, even a miniscule change in the value of a parameter is enough to push the system over the edge into one regime or the other.

When we design a biological circuit, it is important to be aware of bifurcations. Slight deviations in a parameter value that would have only modest effects on system performance away from a bifurcation point can  generate much more drastic changes if they cause the system to bifurcate from one regime to another.

But how can you know whether a given parametrization of the system is close to a bifurcation value or not? A simple way to find out for the positive autoregulation circuit is to determine the number of fixed points in the system and identify the boundary in parameter space between regimes that exhibit one fixed point and those that have three fixed points. 

## Analytically determining the number of fixed points

In this section, we will illustrate how we can mathematically manipulate the ODE for the positive autoregulation system to analytically determine the number of fixed points for a given parametrization. While the approach here is not universal—it cannot be applied directly to any system to analyze its behavior—it illustrates a useful style of reasoning that will help develop intuition for other circuits we encounter.

### Using nondimensionalization to simplify the system

Recall our differential equation for the dynamics of an autoactivator:

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t} = \beta\, \frac{(x/k)^n} {1 + (x/k)^n} - \gamma x,
\end{align}

Here we can identify two variables—$x$ and $t$—and four parameters—$\beta$, $\gamma$, $k$, and $n$. Understanding how the circuit behaves as four parameters are varied is challenging. However, the dynamics do not depend on all four parameters _independently_. In a simpler case, we saw that for unregulated expression, the steady state protein concentration is given by the ratio $\beta/\gamma$. It is the _ratio_ that matters, not the individual values of $\beta$ and $\gamma$. 

The procedure of **nondimensionalization** allows us to reduce the number of independent parameters and still be able to explore the full dynamics. The key idea is that each variable or parameter has some **dimension**. For example, $x$ has dimension of concentration, or number of particles per cubic length, written as

\begin{align}
x = \left[\frac{N}{L^3}\right].
\end{align}

Similarly, the parameter $\gamma$ has dimension of inverse time, or $\gamma = [T^{-1}]$. It is clear that each term in the dynamical equations has dimension of $N/L^3 T$. In general, every term in an equation must have the same dimension; trying to add, for example, a meter to a kilogram is nonsensical.

The nondimensionalization procedure involves rewriting the equations such that every term is **dimensionless**, which means that every term has dimension of unity. This is beneficial for several reasons.

1. It reduces the number of parameters you need to consider. Only dimensionless ratios/products of parameters are considered.
2. It allows comparison of the relative magnitudes of terms in an equation.
3. It provides intuition by identifying which ratios and products of parameters control the dynamics.

There is no one way to nondimensionalize a set of dynamical equations (although [the Buckingham π Theorem](https://en.wikipedia.org/wiki/Buckingham_π_theorem) formalizes how many total dimensionless parameters will be present). We have found the following procedure to be generally effective. Let $\theta$ be one of the variables in a dynamical equation, including space and time. In the above dynamical equations, $\theta$ could be either $x$ or $t$. (There are no spatial variables in this problem. Also, recall that $\beta$, $\gamma$, $k$, and $n$ are system *parameters*, not variables.)

1. For each variable and time, define a dimensionless version (which we will usually mark with a tilde) such that $\theta = \theta_d \tilde{\theta}$, where $\theta$ is some variable or time. The constant $\theta_d$ imparts the dimension on $\theta$ (hence the subscript *d*).
2. Substitute these expressions into the dynamical equations.
3. Rearrange the equations such that every term is dimensionless. As a result, you find ratios and products of the $\theta_d$'s and the system parameters. These ratios are dimensionless and are called *dimensionless parameters*.
4. Choose expressions for the $\theta_d$'s that minimize the number of dimensionless parameters.

As usual, this is best seen by example. We start be defining

\begin{align}
&t = t_d\,\tilde{t},\\[1em]
&x = x_d\,\tilde{x}.
\end{align}

Substituting these into the dynamical equation, we obtain

\begin{align}
\frac{x_d}{t_d}\frac{\mathrm{d}\tilde{x}}{\mathrm{d}\tilde{t}} = \beta\,\frac{(x_d\tilde{x}/k)^{n}}{1 + (x_d\tilde{x}/k)^{n}} - \gamma x_d\tilde{x}.
\end{align}

We now want to make every term dimensionless. Looking at the above expression, we can immediately see that if we set $x_d = k$ then the terms inside the activating Hill term will become dimensionless, yielding

\begin{align}
\frac{k}{t_d}\frac{\mathrm{d}\tilde{x}}{\mathrm{d}\tilde{t}} = \beta\,\frac{\tilde{x}^{n}}{1 + \tilde{x}^{n}} - \gamma k\tilde{x}.
\end{align}

We can now divide both sides of our equation by $k/t_d$ in order to obtain 

\begin{align}
\frac{\mathrm{d}\tilde{x}}{\mathrm{d}\tilde{t}} = \frac{\beta t_d}{k}\,\frac{\tilde{x}^{n}}{1 + \tilde{x}^n} - \gamma t_d \tilde{x}.
\end{align}

In this new form, it is clear that choosing $t_d = 1/\gamma$ will eliminate a parameter from the final term on the right hand side. This makes the prefactor of the Hill activating term equal to $\beta / \gamma k$, which we can determine is a dimensionless quantity. For notational convenience we will denote this term $\tilde{\beta}$. Now we are done! Our dimensionless ODE is then

\begin{align}
\frac{\mathrm{d}\tilde{x}}{\mathrm{d}\tilde{t}} = \tilde{\beta}\,\frac{\tilde{x}^{n}}{1 + \tilde{x}^n} - \tilde{x}.
\end{align}

We now see that the entire nondimensional dynamics of our system can be described using only two parameters, $\tilde{\beta}$ and $n$. 

As an added bonus, nondimensionalization makes clear what dimensionless parameters, which are ratios of dimensional parameters, are important for the dynamics. It is always important to reason about what the dimensionless parameters represent. In this case, we already know that $n$ is the Hill coefficient, which represents the ultrasensitivity of the gene's regulation. We see from its definition that $\tilde{\beta} = \beta/\gamma k$ is the ratio of the gene's unregulated steady state (which we know from Chapter 1 to be $\beta/\gamma$) and the coefficient $k$, the concentration of X at which the activation of its regulated gene is half-maximal. In other words, $\tilde{\beta}$ can be interpreted as a ratio of X's expression level to the level of X necessary to activate itself.

Another way to look at this result is to say that we have chosen meaningful units of time and concentration. We are effectively measuring time in units of the key timescale in the system—the protein lifetime $\gamma^{-1}$—and protein concentrations in terms of the only concentration scale in the problem, the amount required for half-maximal repression, $k$. 

Because tildes can be notionally cumbersome, we will often drop them for convenience, writing redefinition of parameters and variables as, for this example,

\begin{align}
&t \leftarrow \gamma t,\\[1em]
&x \leftarrow x/k,\\[1em]
&\beta \leftarrow \beta/\gamma k,
\end{align}

such that

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t} = \beta\,\frac{x^{n}}{1 + x^n} - x.
\end{align}

Nondimensionalization is often the first step in analyzing a model for circuit behavior, We will apply it again and again throughout this book.

### Analytical methods for exploring steady states

From our analysis above, we found that the system has the following properties.

1. The system always has a fixed point at $x = 0$.
2. The system has either zero, one, or two fixed points with $x > 0$.
3. If the system has zero fixed points with $x > 0$, it is monostable at $x = 0$.
4. If the system has two fixed points with $x > 0$, the greater of the two is stable, as is the fixed point at zero, with the middle fixed point being unstable. This is the **bistable regime.**


If we can find the number of fixed points, we can determine if the system is monostable or bistable. Fortunately, now that we have expressed the system's dynamics using only two parameters, it is easier to analyze.

Finding the fixed points amounts to finding the roots of the equation

\begin{align}
f(x) = \beta \frac{x^n}{1 + x^n} - x = 0,
\end{align}

where $f(x)$ is the right hand side of the dimensionless ODE defined above.

Since we know that $x=0$ is always a fixed point, for the remainder of our analysis, we will therefore assume that $x>0$ as we try to determine if there are any nonzero fixed points. (We ignore fixed points where $x < 0$, since a negative concentration is unphysical.)

With these assumptions, and some basic algebraic operations, we can rewrite the equation $f(x) = 0$ as

\begin{align}
x^n - \beta x^{n-1} + 1 = 0.
\end{align}

We'll call the left hand side of this equation $g(x)$. We are trying to find positive values of $x$ for which $g(x) = 0$. To find the roots, we need to consider different values of $n$ separately.

**The n > 1 regime**

From looking at the form of the $g(x)$ expression, as long as $n > 1$ we can see that in the limit as $x$ approaches $0$, $g(x)$ approaches a value of $1$. Meanwhile, as $x$ gets large, we see that the $x^n$ term will dominate the expression and $g(x)$ will approach infinity.

By inspection, the function $g(x)$ is continuous for $n > 1$, and the limit of $x = 0$ is finite, $g(x)$ must have a finite minimum which occurs at $x = x_\mathrm{min}$. If $g(x_{\mathrm{min}}) > 0$, then we know that $g(x)$ never dips low enough to satisfy $g(x) = 0$, and $x=0$ is our only fixed point. On the other hand, if $g(x_{\mathrm{min}}) < 0$, then we know that there must be at least two nonzero fixed points, since $g(x)$ had to cross the $x$-axis to get from $1$ to $g(x_{\mathrm{min}})$ and must cross it again to get from $g(x_{\mathrm{min}})$ to $+\infty$. 

To make this more clear, in the plot below, we visualize the known limiting behavior at the edges using dark lines and potential behaviors of $g(x)$ in the interior using faint lines. 

In [5]:
def g(x, beta, n):
    return x ** n - beta * x ** (n - 1) + 1


x0 = -3
x1 = 0.6
x_left = np.logspace(-4, x0, 100)
x_right = np.logspace(x1, 1, 100)
x_mid = np.logspace(x0, x1, 400)

# Build plot
p = bokeh.plotting.figure(
    frame_height=275,
    frame_width=275,
    x_axis_label="x",
    y_axis_label="g(x)",
    x_range=[1e-4, 1e1],
    y_range=[-1, 15],
    x_axis_type="log",
)

# Emphasize x axis
p.line([1e-6, 1e2], [0, 0], line_width=1, color="black")

# Plot lines
p.line(x_mid, g(x_mid, 1.5, 2), line_width=2, color="#6baed6")
p.line(
    np.append(x_mid, 7), g(np.append(x_mid, 7), 2.2, 2), line_width=2, color="#6baed6",
)
p.line(x_left, g(x_left, 1.5, 2), line_width=2, color="#08519c")
p.line(x_right, g(x_right, 1.5, 2), line_width=2, color="#08519c")

# Add labels
p.text(
    x=[1.3e-4],
    y=[1.2],
    text=["g(x) → 1 as x → 0"],
    text_align="left",
    text_baseline="bottom",
    text_font_size='8pt',
    text_color="#08519c"
)

p.text(
    x=[3e0],
    y=[14],
    text=["g(x) → ∞ as x → ∞"],
    text_align="right",
    text_baseline="bottom",
    text_font_size='8pt',
    text_color="#08519c"
)

p.text(
    x=[2e0],
    y=[4],
    text=["Does g(x) cross 0?"],
    text_align="right",
    text_baseline="bottom",
    text_color="#6baed6",
    text_font_size='8pt',
)

bokeh.io.show(p)

To find $x_\mathrm{min}$, we can find the value of $x$ for which the derivative of $g(x)$ vanishes and the second derivative is positive; that is where

\begin{align}
&\frac{\mathrm{d}g}{\mathrm{d}x} = nx^{n-1} - \beta(n-1)x^{n-2} = 0,\\[1em]
&\frac{\mathrm{d}^2g}{\mathrm{d}x^2} = n(n-1)x^{n-2} - \beta(n-1)(n-2)x^{n-3} > 0.
\end{align}

For $n > 1$, this is satisfied only if

\begin{align}
x_{\mathrm{min}} = \beta \,\frac{n-1}{n}.
\end{align}

To get the minimal value of $g(x)$, we compute

\begin{align}
g(x_{\mathrm{min}}) &= \left(\beta \,\frac{n-1}{n}\right)^n - \beta \left(\beta \,\frac{n-1}{n}\right)^{n-1} + 1 \\[1em]
&= 1 - \left(n-1\right)^{n-1}\left(\frac{\beta}{n}\right)^n.
\end{align}

Therefore, the condition for $g(x)$ to be less than zero for some values of $x$ is $g(x_\mathrm{min}) < 0$, which, using the above expression and rearranging, is equivalent to

\begin{align}
\beta > n (n-1)^{(1-n)/n}.
\end{align}

In the regime where the above inequality holds, there is a unique minimum of $g(x)$ below zero, and therefore the system has two positive fixed points, in addition to the fixed point at $x = 0$. When $\beta < n (n-1)^{(1-n)/n}$, the unique minimum is above zero, so the $g(x)$ curve never crosses the x-axis, and the fixed point at zero is the only fixed point. So, when $\beta = n (n-1)^{(1-n)/n}$ the system switches from having one to having three fixed points. This line in parameter space represents a **bifurcation**, where the qualitative properties of the system change upon crossing it.

**The n ≤ 1 regime:**

We have not yet considered the $n \le 1$ regime. In an end-of-chapter problem, you can prove that only one stable fixed point exists in this regime, regardless of the values of $\beta$. We will take that as given going forward.

### A bifurcation diagram

We have now analyzed the system fully and shown that the system is monostable for $n\leq 1$. For $n>1$, the system is bistable when $\beta > n(n-1)^{(1-n)/n}$, monostable when $\beta < n(n-1)^{(1-n)/n}$, and undergoes a bifurcation between these two regimes when $\beta = n(n-1)^{(1-n)/n}$. 

Thanks to the fact that we nondimensionalized our system, its bifurcation behavior can be fully explained using only two dimensionless parameters. This allows us to visualize it with the nature of the steady states of the system with a **bifurcation diagram**, shown below.

In [6]:
# Get bifurcation line
n = np.linspace(1, 10, 400)
beta = n * (n - 1) ** ((1 - n) / n)

# Build the plot
p = bokeh.plotting.figure(
    height=300,
    width=400,
    x_axis_label="n",
    y_axis_label="nondimesional β",
    x_range=[0, 10],
    y_range=[0, 3],
    tools=["save"],
)
p.patch(np.append(n, [n[-1], 1]), np.append(beta, [0, 0]), color="lightgray", alpha=0.7)
p.patch([0, 0, 1, 1], [0, 10, 10, 0], color="lightgray", alpha=0.7)
p.patch(np.append([n[-1], 1], n), np.append([3, 3], beta), color="gray", alpha=0.7)
p.line(n, beta, line_width=2, color="black")
p.line([1, 1], [1, 5], line_width=2, color="black")
p.text(x=4.5, y=2.2, text=["bistable"])
p.text(x=3.2, y=0.8, text=["monostable"])

bokeh.io.show(p)

This analysis can help us engineer a system to be bistable. By using a strong ($\beta$ large) and ultrasensitive ($n$ larger than 1, and preferably larger than 2) regulatory system, we can ensure bistability. Further, if we measure the values of our parameters and find that we are near the bifurcation boundary, then we know that small variations in the parameter values can cause a large, qualitative change in the system's behavior. In particular, we can see from the plot above that the regime where $1 < \beta < 2$ and $1 < n < 2$ is a particularly bifurcation-prone regime that we would definitely want to avoid designing our system into if its desired performance is to be reliably bistable.

Because considerations like measurement error and random fluctuations in system components ensure that parameters' values can never be known to perfect confidence, it is important to design a system to lie far away from a bifurcation boundary in order to ensure its robust and reliable performance.

## Bistability enables memory through hysteresis

Bistability's most obvious function is enabling a clonal cell population to differentiate into phenotypically-distinct subpopulations. But a second function is its ability to enable memory of transient signals through a feature known as **hysteresis**. 

Consider the postive autoregulation circuit in its bistable regime, with $x$ at the upper stable fixed point. Now, suppose some transient perturbation occurs. Perhaps the value of $\gamma$ is suddenly increased because the cell has begun dividing at a much faster rate. If this perturbation is large enough to cross the bifurcation boundary, then the system becomes monostable, and $x$ will be driven to the only stable fixed point, at a value closer to what was previously the lower fixed point in the bistable regime. Once the perturbation ends, $\gamma$ returns to its original value, and the system regains bistability. But, if the perturbed value of $x$ lies within the basin of attraction of the lower bistable state, then the system will remain in the lower state. In this scenario, a transient perturbation will have led to a permanent state switch. In this sense the system has a "memory" of the transient event. 

This scenario is illustrated in the series of plots below.

In [7]:
# Parameters
beta = 10
gamma = 1
k = 4
n = 4

# Theretical curves
x = np.linspace(0, 20, 400)
prod = beta * (x/k) ** n / (1 + (x/k) ** n)
removal = gamma * x

# Fixed points
fp_rhs = lambda x: beta / gamma * (x/k) ** n / (1 + (x/k) ** n)
fixed_points = [
    0,
    float(scipy.optimize.fixed_point(fp_rhs, 3)),
    float(scipy.optimize.fixed_point(fp_rhs, 5)),
]
fp3 = fixed_points[2]

# Build plot
p1 = bokeh.plotting.figure(
    frame_height=150,
    frame_width=375,
    x_axis_label="x",
    y_axis_label="production or removal rate",
    title=f"Initial environment. β = {beta}, γ = {gamma} , k = {k}, n = {n}",
    x_range=[-1, 20],
    y_range=[-1.5, 15],
)
p3 = bokeh.plotting.figure(
    frame_height=150,
    frame_width=375,
    x_axis_label="x",
    y_axis_label="production or removal rate",
    title=f"Return to initial environment. β = {beta}, γ = {gamma} , k = {k}, n = {n}",
    x_range=[-1, 20],
    y_range=[-1.5, 15],
)

# Plot production and removal rates
p1.line(x, prod, line_width=2, color="#1f77b4")
p1.line(x, removal, line_width=2, color="orange")
p3.line(x, prod, line_width=2, color="#1f77b4")
p3.line(x, removal, line_width=2, color="orange")

# Plot fixed points
for i, fp in enumerate(fixed_points):
    fill_color = "white" if i % 2 else "black"
    p1.circle(
        [fp], [gamma * fp], color="black", size=10, line_width=2, fill_color=fill_color,
    )
    p3.circle(
        [fp], [gamma * fp], color="black", size=10, line_width=2, fill_color=fill_color,
    )

# Recalculate plot for disturbance condition
gamma = 2.5  # new value

prod = beta * (x/k) ** n / (1 + (x/k) ** n)
removal = gamma * x

# Build plot
p2 = bokeh.plotting.figure(
    frame_height=150,
    frame_width=375,
    x_axis_label="x",
    y_axis_label="production or removal rate",
    title=f"Perturbed environment. β = {beta}, γ = {gamma} , k = {k}, n = {n}",
    x_range=[-1, 20],
    y_range=[-1.5, 15],
)

# Plot production and removal rates
p2.line(x, prod, line_width=2, color="#1f77b4")
p2.line(x, removal, line_width=2, color="orange")

# Plot fixed points
p2.circle(
    [0], [0], color="black", size=10, line_width=2, fill_color="black",
)

# Add commentary
p1.circle(
    [fp3], [fp3], color="gray", size=25, line_width=3, fill_color=None,
)
p1.text(
    x=[10],
    y=[7],
    text=["system is here"],
    text_color="gray",
    text_font_size="10pt",
    text_align="left",
    angle=0,
)
p2.add_layout(
    bokeh.models.Arrow(
        end=bokeh.models.VeeHead(size=15, fill_color="gray", line_color="gray"),
        line_width=4,
        x_start=fp3,
        y_start=fp3 - 0.5,
        x_end=1,
        y_end=1,
        line_color="gray",
    )
)
p2.text(
    x=[5.5],
    y=[3.5],
    text=["system moves here"],
    text_color="gray",
    text_font_size="10pt",
    text_align="left",
    angle=0,
)
p2.circle(
    [0], [0], color="gray", size=25, line_width=3, fill_color=None,
)
p3.text(
    x=[1],
    y=[-1.5],
    text=["system remains here"],
    text_color="gray",
    text_font_size="10pt",
    text_align="left",
    angle=0,
)
p3.circle(
    [0], [0], color="gray", size=25, line_width=3, fill_color=None,
)

# Build layout
hysteresis_layout = bokeh.layouts.gridplot(
    [
        p1,
        bokeh.layouts.Spacer(height=10),
        p2,
        bokeh.layouts.Spacer(height=10),
        p3,
        bokeh.layouts.Spacer(height=10),
    ],
    ncols=1,
)

bokeh.io.show(hysteresis_layout)

This is the principle of hysteresis—a dependence of the system state on its past history. We will now examine how a circuit can be designed to explicitly enable the recording of an external stimulus via hysteresis in a bistable system.

## The toggle switch: a two-gene positive feedback system

In natural circuits, positive feedback loops often include multiple regulators rather than just a single autoregulatory transcription factor. For example, the "genetic switch" in the bacterial virus, or bacteriophage, lambda involves reciprocal repression of the *cro* repressor by the $\lambda$ *cI* repressor, and vice versa (see [Mark Ptashne's classic book, _A genetic switch_](https://www.amazon.com/Genetic-Switch-Third-Lambda-Revisited/dp/0879697164)). This arrangement allows lambda to remain in its dormant "lysogenic" state,  where $\lambda$ *cI* is expressed and represses *cro*, or switch to a "lytic" state in which *cro* is expressed, repressing *cI*. In the lytic phase, te  phage replicates itself and lyses the host cell, enabling lambda to go on to infect other cells. Interestingly, cI is not only a repressor, but also an activator, and it activates its own expression in an ultrasensitive manner due to cooperative binding interactions. Thus, the natural circuit combines a 1-gene positive autoregulation circuit with a 2-gene mutual repression circuit. 

In 2000, [Gardner and Collins](https://doi.org/10.1038/35002131) designed, constructed, and analyzed a synthetic version of this "toggle switch," and showed that it could exhibit bistability. Here is a simplified diagram of the general design:

<div style="width: 350px; margin: auto;">

![toggle](figs/toggle_diagram.png)

</div>

Importantly, they chose to build their circuit using two repressors whose ability to repress their target genes could be inactivated by corresponding small molecule inducers (or in some cases by a temperature shift that inactivated a temperature-sensitive repressor mutant). They could thereby inhibit each of the two repressors, pushing the circuit into a monostable regime, with the other repressor highly expressed. When these input signals were subsequently removed, the circuit returned to a bistable regime, but, due to hysteresis, its expression states remained largely unchanged. In this way, they showed that the toggle circuit could record a minimal 1 bit memory of the last signal that it experienced.

The figure below, featuring data digitized from the paper, shows the toggling behavior. IPTG is an inducer that inhibits repression by LacI, one of the two repressors they used in the toggle. In a control experiment without the toggle architecture, expression of the gene under control of the promoter repressed by LacI grows steadily with increasing IPTG concentration. Conversely, for the toggle architecture, there is a sharp jump in expression at an intermediate IPTG concentration, marking the "switch" of the toggle.

In [8]:
df = pd.read_csv(os.path.join(data_path, "collins_toggle_data.csv"))

p = bokeh.plotting.figure(
    frame_width=300,
    frame_height=200,
    x_axis_label="[IPTG] (mM)",
    y_axis_label="normalized expression",
    x_axis_type="log",
)

colors = ["#1f77b3", "orange"]
for color, (trial, g) in zip(colors, df.groupby("trial")):
    p.circle(
        g["iptg (mM)"], g["normalized gfp expression"], color=color, legend_label=trial
    )
    p.segment(
        source=g,
        x0="iptg (mM)",
        y0="error_low",
        x1="iptg (mM)",
        y1="error_high",
        line_width=2,
        color=color,
        legend_label=trial,
    )

p.legend.location = "top_left"
p.legend.click_policy = "hide"

bokeh.io.show(p)

To analyze this two-repressor feedback loop, we can write down differential equations for the concentrations of the $x$ and $y$ proteins, assuming for simplicity that they share the same Hill coefficient, $n$. For now we will neglect the dynamics associated with using the inducer signals to inactivate a particular repressor. 

\begin{align}
&\frac{dx}{dt} = \frac{\beta_x}{1 + (y/k_y)^n} - \gamma_x x,\\[1em]
&\frac{dy}{dt} = \frac{\beta_y}{1 + (x/k_x)^n} - \gamma_y y.
\end{align}

We nondimensionalize by taking

\begin{align}
&\beta_x \leftarrow \beta_x/\gamma_x k_x,\\[1em]
&\beta_y \leftarrow \beta_y/\gamma_y k_y,\\[1em]
&x \leftarrow x/k_x,\\[1em]
&y \leftarrow y/k_y,\\[1em]
&t \leftarrow \gamma_x t,\\[1em]
&\gamma = \gamma_y/\gamma_x.
\end{align}

As before, this corresponds to measuring each repressor concentration in units of the concentration required to half-maximally repress its target, and measuring time in units of one of the repressor lifetimes. With this, we can now clearly see that the behavior of the system depends only on (a) the strengths of the two promoters, (b) the _relative_ lifetimes of the two proteins, and (c) the sensitivity of the response (Hill coefficient):

\begin{align}
\frac{dx}{dt} &= \frac{\beta_x}{1 + y^n} - x,\\[1em]
\gamma^{-1}\,\frac{dy}{dt} &= \frac{\beta_y}{1 + x^n} - y.
\end{align}

A good next step to analyze a two-dimensional system like this is to compute the **nullclines**, which are defined by setting each of the time derivatives equal to zero:

\begin{align}
x\text{ nullcline: }& x = \frac{\beta_x}{1 + y^n}, \\[1em]
y\text{ nullcline: }& y= \frac{\beta_y}{1 + x^n}.
\end{align}

Wherever these two lines cross, one has a fixed point, since both time derivatives vanish simultaneously. The stability of that fixed point can be determined by **linear stability analysis**, which we will introduce in a later chapter. For now, we will [plot the two nullclines](figs/toggle_nullclines.html) and investigate their behavior by varying the dimensionless parameters $\beta_x$ and $\beta_y$, as well as the Hill coefficient $n$. 

In [9]:
bokeh.io.show(biocircuits.jsplots.toggle_nullclines())

You can investigate the nullclines by playing with the sliders. For $\beta_x = \beta_y = 10$ and $n = 4$, we clearly have three crossings of the nullclines, giving one unstable (the one in the middle) and two stable (the ones on the ends) fixed points. 

Play with the $n$ slider: By moving $n$ down, you can see that bistability persists at lower Hill coefficients. However, it requires a more delicate balancing act, where the values of $\beta_x$ and $\beta_y$ need to be large and close in magnitude. Then, at $n=1$, bistability is lost completely and the system becomes monostable. 

Even at higher Hill coefficients, the strengths of the promoters must still be balanced. If we use widely varying values of $\beta_x$ and $\beta_y$, we again have monostability. You can see this by setting $\beta_x = 10$, $\beta_y = 1.5$, and $n = 5$ with the sliders.

The classic synthetic toggle circuit provides an ideal example of how synthetic biology can be used to design, create, and test simple circuits in living cells. 

### Conclusions: Bistability and hysteresis are powerful features that can be generated through remarkably simple circuits. 

Taken together, our analysis of the positive autoregulatory and the toggle switch revealed some key design principles for bistability and hysteresis:

1. Positive transcriptional feedback loops of 1 or 2 repressors can generate bistability. (Although we did not show it here, the same principles extend to larger feedback loops as well.)
2. Ultrasensitivity within the positive feedback loop is necessary for bistability. However, it is not sufficient, as bistability also requires balance among other parameters. 
3. Bistable systems exhibit hysteresis, which allows cells to permanently switch states in response to a transient input signal. 

Later in the course, we will continue to analyze this system and its stability, learning how to further characterize dynamical systems beyond their nullclines. We will also explore multistable circuit architectures that can scale to produce many stable states. We are only beginning to explore the many fascinating aspects of multistable systems.

## Computing environment

In [10]:
%load_ext watermark
%watermark -v -p numpy,scipy,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 8.1.1

numpy     : 1.20.3
scipy     : 1.7.3
bokeh     : 2.4.2
jupyterlab: 3.3.2



<hr>

## Problems

- [3.1: Modes of bistability](../problems/03/problem_3.1.ipynb)
- [3.2: Controlling an autoactivation circuit](../problems/03/problem_3.2.ipynb)
- [3.3: Autoactivation, bistability, and the importance of leakage](../problems/03/problem_3.3.ipynb)
- [3.4: Modeling inducers in a toggle switch](../problems/03/problem_3.4.ipynb)
- [3.5: Negative autoregulation including mRNA dynamics](../problems/03/problem_3.5.ipynb)